<a href="https://colab.research.google.com/github/saramirabi/Metaheuristic/blob/main/travelsalesman_geneticalgorithms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
######Importing Depedencied#########
from random import randint as rnd
from random import shuffle
import numpy as np
import matplotlib.pyplot as plt
import cv2
#######Setting Parameters######
N_CITIES = 10
WIDTH = 500
HEIGHT = 500
POPULATION_SIZE = 4
EPOCH = 200

######Random City Generator######
def random_city_generator(n, w, h):
    offset = 20
    cities = []
    i = 0
    while i < n:
        city_location = [rnd(offset, w-offset), rnd(offset, h-offset)]
        if city_location not in cities:
            cities.append(city_location)
            i+=1
    return cities


#######Initial Population#####
def init_population(n, ps):
    population_list=[]
    for i in range(ps):
        path = [i for i in range(n)]
        shuffle(path)
        path +=[None]
        population_list.append(path)
    return population_list


#######Cross Over Function#######
def cross_over(population_list, n, p):
    for i in range(p):
        path = population_list[i][:n] + [None]
        population_list.append(path)
    return population_list

def mutation(population_list, n, p):
    length = p * 2
    i = p

    while i <  length:
        cell1= rnd(0, n-1)
        cell2= rnd(0, n-1)

        if cell1 != cell2:

            population_list[i][cell1], population_list[i][cell2] = population_list[i][cell2], population_list[i][cell1]
            i+=1

    return population_list

def path_cordinates(cities_locations, path):
    cordinates = []
    for i in path:
        cordinates.append(cities_locations[i])
    return cordinates

def euclidean_distance(path):
    distance = 0
    for i in range(len(path)-1):
        distance += np.sqrt((path[i][0]- path[i+1][0])**2 + (path[i][1]-path[i+1][1])**2)
    return distance
def fitness(population_list, n , location_list):
    for i in range(len(population_list)):
        if population_list[i][-1] == None:
            current_path = path_cordinates(location_list, population_list[i][:n]+[population_list[i][0]])
            d = euclidean_distance(current_path)
            population_list[i][n]= d
    return population_list


###Sorter Function
def sorter(population_list, k):
    population_list.sort(key = lambda x: x[k])
    return population_list

def draw_cities(img, cities_locations):
    for x,y in cities_locations:
        img = cv2.circle(img, (x,y), 6, (0,0,255), -1)
    return img

def draw_path(img, path, color):
    for i in range(len(path)-1):
        img = cv2.line(img, path[i] , path[i+1], color, 2)
    return img

if __name__ == "__main__":
    cities_locations = random_city_generator(N_CITIES, WIDTH, HEIGHT)
    #print(cities_locations)
    current_population = init_population(N_CITIES, POPULATION_SIZE)
    #print(current_population)

    for i in range(1, EPOCH+1):
        current_population = cross_over(current_population, N_CITIES, POPULATION_SIZE)
        #print(current_population)
        #for i in current_population:
            #print(i)
        #print('******')
        current_population = mutation(current_population, N_CITIES, POPULATION_SIZE)
        #for i in current_population:
            #print(i)
        current_population = fitness(current_population, N_CITIES, cities_locations)
        current_population = sorter(current_population, N_CITIES)
        #for i in current_population:
            #print(i)
        current_population = current_population[:POPULATION_SIZE]
        #print("Best Path and Distance so far:", current_population[0])

    else:
        print("Best FOUND SOLUTION:", current_population[0])
        area = np.full((WIDTH, HEIGHT, 3), 255, np.int16)
        area = draw_cities(area, cities_locations)
        current_path = path_cordinates(cities_locations, current_population[0][:N_CITIES])
        current_path +=[current_path[0]]
        area = draw_path(area, current_path, (187, 134, 192))
        print(current_path)
        plt.imshow(area)
        plt.grid()
        plt.show()